# Convergence on local interaction model


## Loading modules

In [1]:
using QSWalk
using LightGraphs # for graph functions 
using GraphPlot # for ploting graphs 
using LinearAlgebra # for linear algebra utilities

## Numerical proof of unique stationary state

Basic parameters. We use Erdős–Rényi model to generate a directed graph. Strongly connected graphs have unique stationary state. Note the Hamiltonian is chosen to be the adjacency matrix of the underlying graph.

In [5]:
# number of nodes
dim = 8
# smaller vale of 'prob' can be used to geneate graphs which are not strongly connected
prob = 0.6
digraph = erdos_renyi(dim, prob, is_directed=true)
graph = Graph(digraph)

adj_digraph = Matrix(adjacency_matrix(digraph, dir=:in))
adj_graph = Matrix(adjacency_matrix(graph))
time = 100.

println("The graph is strongly connected: $(is_strongly_connected(digraph))")
#gplot(digraph)

The graph is strongly connected: true


Now we can calcuate the lindbladian and the subgroup generator.

In [8]:
lind = local_lind(adj_digraph)
evo_gen = evolve_generator(adj_graph, lind);

The sufficient and necessary condition for the convergence of quantum stochastic evolution is that the null-space is  one-dimensional. Note for large matrices *eigs* may be a better option.

In [10]:
eigvals(evo_gen)

64-element Vector{ComplexF64}:
    -6.114392293005795 - 9.558110425025025e-15im
    -5.771712789357804 - 2.2660808655481804e-15im
   -5.5761226383045805 + 0.08649747420851855im
    -5.576122638304544 - 0.08649747420851714im
    -5.099019074685129 - 7.0558577542407965im
   -5.0990190746851285 + 7.055857754240804im
    -4.991356860078513 + 6.939701942399862im
    -4.991356860078479 - 6.939701942399847im
    -4.846771868939463 - 1.068697449272354im
    -4.846771868939461 + 1.0686974492723622im
    -4.833606716293294 - 0.9808851327719176im
    -4.833606716293292 + 0.9808851327719186im
    -4.760272308068932 - 1.038421380760541im
                       ⋮
   -3.4657590362395645 + 2.4364361852094967im
   -3.4657590362395347 - 2.4364361852094905im
    -3.439540704615754 - 2.0278527189490823im
    -3.439540704615728 + 2.027852718949071im
   -3.3894503229371735 - 2.2423327480568847im
   -3.3894503229371615 + 2.242332748056888im
   -3.1955962478361153 + 0.19511365239812847im
    -3.19559624783611

In [9]:
null_dim = count(x->abs(x)<1e-5, eigvals(evo_gen))
println("Dimensionality of null-space of the evolution operator: $null_dim")

Dimensionality of null-space of the evolution operator: 1


This allows efficient stationary state generation. Note that the trace may differ from one, as the eigenstate is normalized according to different norm.

In [11]:
eigendecomposition = eigen(evo_gen)
zeroindex = findfirst(x -> abs(x)<=1.e-5, eigendecomposition.values)
stationary_state = unres(vec(eigendecomposition.vectors[:, zeroindex]))

println("Trace of stationary state: $(sum(diag(stationary_state)))")
stationary_state /= sum(diag(stationary_state))
println("Trace of stationary state after the normalization: $(sum(diag(stationary_state)))")

Trace of stationary state: 2.5555934493694465 + 3.462508058049707e-15im
Trace of stationary state after the normalization: 1.0 - 1.232595164407831e-32im


## Convergence

Since the stationary state is unique, all of states converge to it. We can show check this by taking three different states. Note, that for larger density states larger times of evolution might be required to achieve the convergence.

In [10]:
rhoinit1 = proj(1, dim)
rhoinit2 = proj(3, dim)
rhoinit3 = Diagonal(I,dim)/dim

8×8 Diagonal{Float64, Vector{Float64}}:
 0.125   ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅ 
  ⋅     0.125   ⋅      ⋅      ⋅      ⋅      ⋅      ⋅ 
  ⋅      ⋅     0.125   ⋅      ⋅      ⋅      ⋅      ⋅ 
  ⋅      ⋅      ⋅     0.125   ⋅      ⋅      ⋅      ⋅ 
  ⋅      ⋅      ⋅      ⋅     0.125   ⋅      ⋅      ⋅ 
  ⋅      ⋅      ⋅      ⋅      ⋅     0.125   ⋅      ⋅ 
  ⋅      ⋅      ⋅      ⋅      ⋅      ⋅     0.125   ⋅ 
  ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅     0.125

Since we apply the same evolution for all of the initial states, it is more efficient to calulate exponent once.

In [11]:
U = evolve_operator(evo_gen, time)
rho1 = evolve(U, rhoinit1)
rho2 = evolve(U, rhoinit2)
rho3 = evolve(U, rhoinit3);

In order to show those states are essentialy the same we can calulate the norm of the difference.

In [12]:
println(norm(rho1-rho2))
println(norm(rho2-rho3))
println(norm(stationary_state-rho3))

8.041235704745722e-18
1.166842690259138e-16
2.6787735595000584e-14
